# Домашнее задание к лекции «Понятие класса»

## Задание 1

Напишите функцию, которая возвращает название валюты (поле `Name`) с максимальным значением курса с помощью сервиса www.cbr-xml-daily.ru...ly_json.js

### Решение Задания 1

In [1]:
import requests

def get_currency_name_with_max_rate():
    """
    Функция возвращает название валюты с максимальным значением курса
    с помощью сервиса www.cbr-xml-daily.ru/daily_json.js

    Возвращаемое значение:
    ---------------------
    max_rate_currency (str): Название валюты с максимальным курсом.
    """
    url = "http://www.cbr-xml-daily.ru/daily_json.js"
    response = requests.get(url)
    data = response.json()

    valute = data.get("Valute")
    max_rate = 0
    max_rate_currency = ""

    for currency, info in valute.items():
        rate = info.get("Value")
        if rate > max_rate:
            max_rate = rate
            max_rate_currency = info.get("Name")

    return max_rate_currency

currency_name = get_currency_name_with_max_rate()
print(currency_name)

СДР (специальные права заимствования)


1. Импорт библиотеки для HTTP запросов.

```
import requests
```

2. Объявление функции `get_currency_name_with_max_rate()`, которая возвращает название валюты с максимальным курсом:
```
def get_currency_name_with_max_rate():
```

3. Внутри функции задается URL сервиса, который предоставляет данные о курсах валют:
```
url = "http://www.cbr-xml-daily.ru/daily_json.js"
```

4. Выполняется HTTP GET запрос к этому URL и результат сохраняется в переменной `response`:
```
response = requests.get(url)
```

5. Текст ответа от сервиса преобразуется из формата JSON в Python-словарь с помощью метода `.json()` и сохраняется в переменной `data`:
```
data = response.json()
```

6. Из данных выбирается раздел `"Valute"`, который содержит информацию о всех валютах, и сохраняется в переменной `valute`:
```
valute = data.get("Valute")
```

7. Задаются начальные значения для переменных `max_rate` (максимальный курс) и `max_rate_currency` (валюта с максимальным курсом):
```
max_rate = 0
max_rate_currency = ""
```

8. Для каждой валюты в `valute` выполняется следующее:
   
   a. Из информации о валюте извлекается текущий курс и сохраняется в `rate`:
    ```
   rate = info.get("Value")
    ```
   b. Если текущий курс больше `max_rate`, тогда `max_rate` и `max_rate_currency` обновляются новыми значениями:
   ```
   if rate > max_rate:
       max_rate = rate
       max_rate_currency = info.get("Name")
   ```


9. После перебора всех валют функция возвращает название валюты с максимальным курсом:
```
return max_rate_currency
```

10. Затем функция вызывается, результат сохраняется в `currency_name` и выводится на экран:
```
currency_name = get_currency_name_with_max_rate()
print(currency_name)
```

## Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

### Решение Задания 2

За основу берем код определяющий класс `Rate` из лекции "Понятие класса", который содержит методы для получения информации о курсах валют.

Основные методы класса `Rate`:

- `__init__`: Конструктор класса, который принимает один аргумент - `format_`, который определяет формат возвращаемых данных. По умолчанию установлено значение 'value'.
- `exchange_rates`: Совершает GET-запрос к сервису с курсами валют, получает и возвращает данные о валютах в формате JSON.
- `make_format`: Возвращает информацию о валюте, указанной аргументом `currency`. Если в `self.format` задано 'full', возвращается полная информация о валюте, в противном случае возвращается только текущий курс.
- `eur`, `usd`, `brl`: Эти методы возвращают курс соответствующей валюты на сегодня в формате, заданном в `self.format`.


Чтобы добавить параметр `diff` и изменить поведение методов `eur`, `usd` и `brl`, нужно сделать следующее:

- Добавим `diff` в конструктор и установим его значение по умолчанию в False:
   ```
   def __init__(self, format_='value', diff=False):
       self.format = format_
       self.diff = diff
   ```


- Изменим метод `make_format`, чтобы он возвращал разницу в курсах, если `self.diff` равно True. Вы можете вычислить разницу, отняв предыдущее значение курса от текущего (разнице может быть отрицательной, если курс упал):
   ```
   if self.format == 'value':
       if self.diff:
           return response[currency]['Value'] - response[currency]['Previous']
       else:
           return response[currency]['Value']
   ```


Таким образом, при установке `diff` в True методы `eur`, `usd` и `brl` будут возвращать изменение в курсе валюты вместо самого курса. Если информация запрашивается в формате 'full', значение `diff` не будет приниматься во внимание (это указано в условии задачи).

Ниже приведен вид измененного класса `Rate`:

In [ ]:
import requests

class Rate:
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff

    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах.
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']

    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full';
        - текущий курс или его изменение при self.format = 'value'.
        """
        response = self.exchange_rates()

        if currency in response:
            if self.format == 'full':
                return response[currency]
            if self.format == 'value':
                if self.diff:
                    return response[currency]['Value'] - response[currency]['Previous']
                else:
                    return response[currency]['Value']
        return 'Error'

    def eur(self):
        """
        Возвращает курс евро на сегодня в формате self.format при diff=False
        или изменение курса при diff=True.
        """
        return self.make_format('EUR')

    def usd(self):
        """
        Возвращает курс доллара на сегодня в формате self.format при diff=False
        или изменение курса при diff=True.
        """
        return self.make_format('USD')

    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')


In [ ]:
# Если `format_` установлен в 'value' (что является значением по умолчанию), этот код выведет текущий курс доллара США.
rate = Rate()
print(rate.make_format('USD'))

97.0018


In [ ]:
# Если `diff` установлен в true, то выведет изменение курса по сравнению с предыдущим днем:
rate = Rate(diff=True)
print(rate.make_format('USD'))

0.5018000000000029


In [ ]:
# Если `format_` установлен в 'full', `make_format` вернет полный набор доступной информации о валюте,
# включая ее название, номинал, текущий курс и предыдущий курс:
rate = Rate(format_='full')
print(rate.make_format('USD'))

{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 97.0018, 'Previous': 96.5}


## Задание 3

Напишите класс `Designer`, который учитывает количество международных премий. Подсказки в коде занятия (“Ноутбук к лекциям «Понятие класса» + презентация”, zip-файл “Используемый ноутбук к лекциям «Понятие класса»).

Комментарий по классу `Designer` такой:
Напишите класс `Designer`, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс `Designer` пишется по аналогии с классом `Developer` из материалов занятия.

### Решение Задания 3

In [ ]:
# Выполняем класс Employee из материалов занятия «Понятие класса»
class Employee:
    def __init__(self, name, seniority):
        """
        Инициализирует объект Employee.

        Аргументы:
        - name (str): Имя сотрудника.
        - seniority (int): Уровень стажа сотрудника.
        """
        self.name = name
        self.seniority = seniority
        self.grade = 1

    def grade_up(self):
        """Повышает уровень сотрудника."""
        self.grade += 1

    def publish_grade(self):
        """Публикует результаты аккредитации сотрудников."""
        print(self.name, self.grade)

    def check_if_it_is_time_for_upgrade(self):
        pass

In [ ]:
# Пишем класс Designer наследуя методы класса Employee
class Designer(Employee):
    def __init__(self, name, awards):
        """
        Инициализирует объект Designer.

        Аргументы:
        - name (str): Имя дизайнера.
        - awards (int): Количество наград, полученных дизайнером.
        """
        super().__init__(name, awards)
        self.awards = awards

    def check_if_it_is_time_for_upgrade(self):
        """Проверяет, настало ли время повышения уровня дизайнера."""
        self.awards += 2

        if self.awards % 7 == 0:
            self.grade_up()

        return self.publish_grade()

In [ ]:
elena = Designer('Елена', 4)

for i in range(15):
    elena.check_if_it_is_time_for_upgrade()
    print(elena.__dict__)

Елена 1
{'name': 'Елена', 'seniority': 4, 'grade': 1, 'awards': 6}
Елена 1
{'name': 'Елена', 'seniority': 4, 'grade': 1, 'awards': 8}
Елена 1
{'name': 'Елена', 'seniority': 4, 'grade': 1, 'awards': 10}
Елена 1
{'name': 'Елена', 'seniority': 4, 'grade': 1, 'awards': 12}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 14}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 16}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 18}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 20}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 22}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 24}
Елена 2
{'name': 'Елена', 'seniority': 4, 'grade': 2, 'awards': 26}
Елена 3
{'name': 'Елена', 'seniority': 4, 'grade': 3, 'awards': 28}
Елена 3
{'name': 'Елена', 'seniority': 4, 'grade': 3, 'awards': 30}
Елена 3
{'name': 'Елена', 'seniority': 4, 'grade': 3, 'awards': 32}
Елена 3
{'name': 'Елена', 'seniority': 4, 'grade':


Объекты класса Designer принимают в качестве аргументов `name` (имя) и `awards` (полученные премии). Инициализация объекта данного класса происходит через метод `__init__()`, который принимает эти два аргумента, дополнительно вызывая с помощью функции `super()` метод `__init__()` базового класса и передавая туда эти же аргументы. Это позволяет сохранить и использовать в дальнейшем функционал, уже описанный в классе Employee.

`self.awards = awards` задает количество премий, которые получил дизайнер.

У класса Designer есть метод `check_if_it_is_time_for_upgrade()`, который проверяет, не пора ли повысить грейд дизайнера. Этот метод при каждом вызове увеличивает количество премий дизайнера `self.awards` на `2`. Затем проверяет, делится ли общее количество премий на `7` без остатка. Если условие выполняется, то с помощью метода `self.grade_up()` повышается "грейд" (уровень) дизайнера. На выходе метод `check_if_it_is_time_for_upgrade()` вызывает метод `self.publish_grade()`, публикуя текущий грейд дизайнера.

Метод `self.grade_up()` предполагается описанным в базовом классе `Employee` и отвечает за повышение "грейда". Метод `self.publish_grade()` также предполагается описанным в `Employee` и отвечает за публикацию текущего грейда сотрудника.

Таким образом, класс Designer использует механизмы наследования для повторного использования кода из класса Employeчество полученных наград и их использование для определения момента повышения грейда.